In [1]:
# IMPORTS
from manufacturing_company.src.common.const import *
from enron.src.common.const import *
from manufacturing_company.src.classification_algorithms.standard_classification import classification
from enron.src.classification_algorithms.assign_management_levels import assign_management_levels
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from manufacturing_company.src.logs.standard_classification_logger import StandardClassificationLogger

/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [4]:
# CONST
df_positions = pd.read_csv(FILE_FLATTEN_HIERARCHY, sep=';', index_col=ID)
df_positions = df_positions[[FLATTEN_POSITION]]
df_positions.columns = [POSITION]

levels = 2

In [5]:
# DECISION TREE
logger = StandardClassificationLogger('enron', levels, DecisionTreeClassifier)

for month in range(1, SIZE + 1):
    df_features = pd.read_csv(FILE_FEATURES.format(month), sep=';', index_col=ID)
    df_features = assign_management_levels(levels, df_features, df_positions)

    classification(df_features, DecisionTreeClassifier, decision_tree_params, 'f1_macro', logger, month)


/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaco

/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaco

/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


...

/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/mateusz/anaco

In [3]:
def decision_tree_params(feature_nr):
    max_depth = np.arange(100) + 1
    max_features = list(range(1, feature_nr))
    return {'model__max_depth': max_depth, 'model__max_features': max_features}
    # max_depth = [3]
    # return {'model__max_depth': max_depth}

In [ ]:
def random_forest_params(feature_nr):
    n_estimators = [2, 3, 5, 10, 15, 20, 40, 60, 80, 100]
    max_depth = [1, 2, 3, 4, 5, 10, 15, 20, 25]
    max_features = list(range(1, feature_nr))
    return {'model__n_estimators': n_estimators, 
            'model__max_depth': max_depth, 
            'model__max_features': max_features}

In [13]:
df_features[[POSITION]].head()

,ManagementLevel
ID,
42,1
38,1
65,1
37,1
12,1


In [16]:
df_features.loc[df_features.index.isin(df_positions.index), POSITION]

ID
42     1
38     1
65     1
37     1
12     1
22     1
160    1
107    1
96     1
51     1
17     1
14     1
95     1
27     1
146    1
34     1
73     1
63     1
67     1
94     1
88     1
39     1
118    1
21     1
33     1
158    1
153    1
5      1
90     1
82     1
      ..
92     1
3      1
116    1
10     1
122    1
119    1
86     1
64     1
129    1
49     1
149    1
32     1
162    1
77     1
23     1
41     1
48     1
79     1
44     1
13     1
30     1
131    1
152    1
91     1
142    1
76     1
55     1
144    1
40     1
174    1
Name: ManagementLevel, Length: 99, dtype: int64

In [20]:
df_positions[df_positions.index.isin(df_features.index)]

,ManagementLevel
ID,
1,1
3,2
4,1
5,1
6,1
7,3
10,3
12,1
13,3


In [6]:
df_features[POSITION]

ID
42     1
38     1
65     1
37     1
12     1
22     1
160    1
107    1
96     1
51     1
17     1
14     1
95     1
27     1
146    1
34     1
73     1
63     1
67     1
94     1
88     1
39     1
118    1
21     1
33     1
158    1
153    1
5      1
90     1
82     1
      ..
92     1
3      1
116    1
10     1
122    1
119    1
86     1
64     1
129    1
49     1
149    1
32     1
162    1
77     1
23     1
41     1
48     1
79     1
44     1
13     1
30     1
131    1
152    1
91     1
142    1
76     1
55     1
144    1
40     1
174    1
Name: ManagementLevel, Length: 99, dtype: int64